# 量子过程层析

*版权所有 (c) 2022 百度量子计算研究所，保留所有权利。*

## 内容概要
在迈向大规模量子计算的道路上，一个核心挑战是如何精确操作量子比特。那么一个能够高效精确刻画量子操作过程的方法就显得十分重要。
量子过程层析（Quantum Process Tomography, QPT）是一种可以通过测量来精确刻画量子过程具体表达形式的方法。
本教程将介绍量子过程层析的理论及其在[百度量子平台](https://quantum.baidu.com/)上的实践。

## 理论

### Pauli Transfer Matrix 表达
一个 $n$ 量子比特的量子信道 $\mathcal{N}$ 可以用 Pauli Transfer Matrix（PTM）来表达为 $[\mathcal{N}]$ ，它的元素定义如下：

$$
    [\mathcal{N}]_{ij} = \textrm{Tr} \left[ P_i \mathcal{N} (P_j) \right],
$$

其中
$$
    P_i, P_j \in \left\{ I / \sqrt{2}, X / \sqrt{2},  Y / \sqrt{2},  Z / \sqrt{2} \right\}^{\otimes n}.
$$

记 $\textrm{Tr} \left[ P_i \mathcal{N} (P_j) \right] = \langle\langle i | [\mathcal{N}] | j \rangle\rangle$，$|i\rangle\rangle,\;|j\rangle\rangle$ 是 PTM 表达下 $P_i,\;P_j$ 对应的基，则我们可以用一个 $4^n \times 4^n$ 的矩阵来刻画一个量子信道

$$
\begin{align}
    [\mathcal{N}] &= \left( \sum_i | i \rangle\rangle \langle\langle i | \right) [\mathcal{N}] \left( \sum_j | j \rangle\rangle \langle\langle j | \right) \\
    &= \sum_{ij} | i \rangle\rangle \langle\langle i | [\mathcal{N}]| j \rangle\rangle \langle\langle j |.
\end{align}
$$



### 过程层析方法

与量子态层析（Quantum State Tomography, QST）相似，我们可以通过测量近似获得 $[\mathcal{N}]$ 的每个元素。我们需要准备一组完备测量算符 $\left\{ E_m \right\}_{m=1}^M(M \geq d^2)$ 和一组完备量子态 $\left\{ \rho_n \right\}_{n=1}^N(N \geq d^2)$，其中 $d = 2^n$ 。因此，我们一共可以得到 $M \times N$ 个期望值（或称为概率值）：

$$
    \forall m \in [M], n \in [N], p_{mn} := \langle\langle E_m | [\mathcal{N}] | \rho_n \rangle\rangle = \textrm{Tr} \left[ E_m \mathcal{N}(\rho_n) \right].
$$

在其中插入完备的 Pauli 基，则有

$$
\begin{align}
    p_{mn} &= \langle\langle E_m | [\mathcal{N}] | \rho_n \rangle\rangle \\
    &= \langle\langle E_m | \left( \sum_{j=1}^{d^2} | j \rangle\rangle \langle\langle j | \right) [\mathcal{N}] \left( \sum_{k=1}^{d^2} | k \rangle\rangle \langle\langle k | \right) | \rho_n \rangle\rangle \\
    &= \sum_{jk} \langle\langle E_m | j \rangle\rangle \langle\langle j | [\mathcal{N}] | k \rangle\rangle \langle\langle k | \rho_n \rangle\rangle,
\end{align}
$$

其中 $\langle\langle E_m | j \rangle\rangle$ 是 $E_m$ 在 Pauli 基下展开的第 $j$ 个元素，$\langle\langle k | \rho_n \rangle\rangle$ 是 $\rho_n$ 在 Pauli 基下展开的第 $k$ 个元素。
因为测量算符 $\left\{ E_m \right\}$ 是实验前选取的，我们可以定义一个大小为 $M \times d^2$ 的矩阵 $\mathfrak{M}$ 来表示：

$$
\begin{align}
  \mathfrak{M} = \begin{pmatrix}
    \langle \langle E_1 | 1 \rangle \rangle & \langle \langle E_1 | 2 \rangle \rangle & \cdots & \langle \langle E_1 | d^2 \rangle \rangle \\
    \langle \langle E_2 | 1 \rangle \rangle & \langle \langle E_2 | 2 \rangle \rangle & \cdots & \langle \langle E_2 | d^2 \rangle \rangle \\
    \vdots & \vdots & \ddots & \vdots \\
    \langle \langle E_M | 1 \rangle \rangle & \langle \langle E_M | 2 \rangle \rangle & \cdots & \langle \langle E_M | d^2 \rangle \rangle \\
  \end{pmatrix}.
\end{align}
$$

同理，$\left\{ \rho_n \right\}$ 也可以用一个 $N \times d^2$ 的矩阵 $\mathfrak{P}$ 来表示。
与 QST 相同的是，我们可以直接测量得到 $p_{mn}$ 的估计值 $\hat{p_{mn}}$ ，如下图所示：
 
![QPT](./figures/qpt-circuit.png "图1：我们通过构造所示电路来获取估计值。")
 
因此我们的问题转换为求解下式中的 $\left[\mathcal{N}\right]$ ：

$$
\begin{align}
  \mathbf{P} = \mathfrak{M} \left[\mathcal{N}\right] \mathfrak{P}^T,
\end{align}
$$

其中 $\mathbf{P}$ 的元素为我们测量得到估计值，即 $P_{mn} = p_{mn}$。
类似地，我们可以选择直接求逆的方式进行求解，也可以利用最小二乘法。
最后我们来回顾一下整个流程：
 
![QPT](./figures/qpt-chart.png "图2：我们按照所示流程图来完成量子过程层析。")
 

## 实践

下面我们将演示如何对一个 CNOT 门进行量子门层析。

首先，我们导入一些必要的包。

In [14]:
import numpy as np
import QCompute
import qcompute_qep.tomography as tomography
import qcompute_qep.quantum.pauli as pauli
import qcompute_qep.utils.circuit
import qcompute_qep.quantum.metrics as metrics

然后，我们搭建一个 CNOT 门。

In [ ]:
qp = QCompute.QEnv()  # qp is short for "quantum program", instance of QProgram
qp.Q.createList(2)

# Manually decompose the CNOT gate using the CZ gate, where CNOT: q1 -> q0
QCompute.H(qp.Q[0])
QCompute.CZ(qp.Q[1], qp.Q[0])
QCompute.H(qp.Q[0])


 
![QPT](./figures/qpt-CNOT-example.png "图3：我们将 CNOT 门作为层析对象。")
 
我们生成如上电路，在进行量子门层析前，我们先求解 CNOT 理想情况下的 PTM 作为参考。

In [ ]:
ideal_cnot = qcompute_qep.utils.circuit.circuit_to_unitary(qp)
ideal_ptm = pauli.unitary_to_ptm(ideal_cnot).data
print(ideal_ptm)

接下来我们需要选择量子计算机的类型，可以选择模拟器或量子计算机真机，这里我们选择的是百度量子真机。剩下的步骤就很简单了，只需要调用我们的 ProcessTomography 模块则可完成真实量子信道的层析，和 QST 类似，我们选择求逆的方式对量子信道的 PTM 进行求解。

In [ ]:
# Please login the "Quantum Leaf" platform (https://quantum-hub.baidu.com/) to get Token
QCompute.Define.hubToken = "Token"

qc = QCompute.BackendName.CloudBaiduQPUQian
# Initialize a ProcessTomography instance
st = tomography.ProcessTomography()
# Call the tomography procedure and obtain the noisy CZ gate
noisy_ptm = st.fit(qp, qc, prep_basis='Pauli', meas_basis='Pauli', method='inverse', shots=8192, ptm=True)


最后，我们可以对结果进行分析，然后可视化我们层析出的 PTM 并和理想情况下进行对比。

In [ ]:
print("****** The average gate fidelity between these two PTMs is: {}".format(
        metrics.average_gate_fidelity(ideal_ptm, noisy_ptm)))

# Visualize these PTMs
diff_ptm = ideal_ptm - noisy_ptm
tomography.compare_process_ptm(ptms=[ideal_ptm, noisy_ptm.data, diff_ptm],
                               show_labels=True)



![QPT](./figures/qpt-output.png "图4：我们对测试结果进行可视化。")


## 总结

本教程旨在介绍 QPT 的原理以及如何在[百度量子平台](https://quantum.baidu.com/)上实现。

## 参考资料
[1] Greenbaum, Daniel. "Introduction to quantum gate set tomography." [arXiv](https://arxiv.org/abs/1509.02921) preprint arXiv:1509.02921 (2015).